# Front Tracking: Rarefaction Waves and Wave Interactions

This notebook demonstrates rarefaction waves and wave interactions in the exact analytical front-tracking solver for advection with nonlinear (Freundlich) sorption.

## Overview

The front-tracking method solves the advection-sorption equation exactly using:
- **Analytical wave tracking**: Characteristics, shocks, and rarefaction waves
- **Event-driven simulation**: No time stepping, only wave interactions  
- **Machine precision**: No numerical tolerances or approximations

## When Do Rarefactions Form?

| Property | Favorable ($n > 1$) | Unfavorable ($n < 1$) |
|----------|---------------------|------------------------|
| **Retardation** | High C → Low R (FASTER) | High C → High R (SLOWER) |
| **Rarefaction from** | Concentration DECREASES | Concentration INCREASES |
| **Shock from** | Concentration INCREASES | Concentration DECREASES |
| **Example** | Pulse (0→10→0): rarefaction on fall | Dip (10→2→10): rarefaction on leading edge |

### Mathematical Foundation

For Freundlich sorption $s(C) = k_f C^{1/n}$:
- **Retardation**: $R(C) = 1 + \frac{\rho_b k_f}{\theta n} C^{(1/n)-1}$
- **Characteristic velocity**: $\lambda(C) = Q/R(C)$
- **Shock velocity**: $s = \frac{Q(C_R - C_L)}{C_{\text{total}}(C_R) - C_{\text{total}}(C_L)}$

Rarefaction waves follow a **self-similar solution**: $R(C) = \frac{Q(t - t_0)}{V - V_0}$, solved analytically with no numerical iteration.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from gwtransport.advection import infiltration_to_extraction_front_tracking_detailed
from gwtransport.fronttracking.plot import (
    plot_front_tracking_summary,
    plot_sorption_comparison,
)
from gwtransport.fronttracking.validation import verify_physics
from gwtransport.utils import compute_time_edges

# Configure plotting
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 10

## Example 1: Concentration Pulse with Favorable Sorption

A concentration pulse ($0 \to 10 \to 0$) with favorable sorption ($n > 1$) demonstrates both wave types:
- **Rising edge** ($0 \to 10$): **Shock wave** (compression - fast 10 catches slow 0)
- **Falling edge** ($10 \to 0$): **Rarefaction wave** (expansion - smooth transition)

### Physical Interpretation
- High concentration ($C=10$) travels FAST (low retardation)
- Rising edge compresses into a shock
- Falling edge spreads out into a rarefaction fan
- Breakthrough: Sharp rise, smooth fall

In [2]:
# Example 1 Setup
tedges_ex1 = pd.date_range(start="2020-01-01", periods=100, freq="D")

# Pulse: 0 → 10 → 0 (shock on rise, rarefaction on fall)
cin_ex1 = np.zeros(len(tedges_ex1) - 1)
cin_ex1[10:40] = 10.0  # Pulse from day 10 to 40

# Aquifer properties
flow_ex1 = np.full(len(tedges_ex1) - 1, 100.0)  # m³/day
aquifer_pore_volume_ex1 = 200.0  # m³

# Freundlich sorption (favorable: n > 1)
freundlich_k_ex1 = 0.01  # (m³/kg)^(1/n)
freundlich_n_ex1 = 2.0  # Favorable sorption
bulk_density_ex1 = 1500.0  # kg/m³
porosity_ex1 = 0.3

# Output grid
cout_tedges_ex1 = pd.date_range(start=tedges_ex1[0], periods=1350, freq="D")

print("Example 1: Concentration Pulse (Favorable Sorption)")
print("  Inlet: 0 → 10 (day 10) → 0 (day 40)")
print(f"  Freundlich: n={freundlich_n_ex1}, k={freundlich_k_ex1}")
print(f"  Pore volume: {aquifer_pore_volume_ex1} m³, Flow: {flow_ex1[0]} m³/day")

Example 1: Concentration Pulse (Favorable Sorption)
  Inlet: 0 → 10 (day 10) → 0 (day 40)
  Freundlich: n=2.0, k=0.01
  Pore volume: 200.0 m³, Flow: 100.0 m³/day


In [ ]:
# Run simulation
cout_ex1, structure_ex1 = infiltration_to_extraction_front_tracking_detailed(
    cin=cin_ex1,
    flow=flow_ex1,
    tedges=tedges_ex1,
    cout_tedges=cout_tedges_ex1,
    aquifer_pore_volumes=aquifer_pore_volume_ex1,
    freundlich_k=freundlich_k_ex1,
    freundlich_n=freundlich_n_ex1,
    bulk_density=bulk_density_ex1,
    porosity=porosity_ex1,
)

# Print summary
print(
    f"\nResults: {structure_ex1[0]['n_events']} events | "
    f"{structure_ex1[0]['n_shocks']} shocks | "
    f"{structure_ex1[0]['n_rarefactions']} rarefactions | "
    f"First arrival: {structure_ex1[0]['t_first_arrival']:.1f} days"
)

# Verify physics (includes mass balance check using exact analytical spatial integration)
results_ex1 = verify_physics(structure_ex1, cout_ex1, cout_tedges_ex1, cin_ex1, verbose=True)
print(f"\n{results_ex1['summary']}")

# Create comprehensive visualization

axes_ex1 = plot_front_tracking_summary(
    structure_ex1,
    tedges_ex1,
    cin_ex1,
    cout_tedges_ex1,
    cout_ex1,
    t_max=150,
    title="Example 1: Pulse with Favorable Sorption (n=2.0)",
)
plt.show()

print("\nKey Features:")
print("  • V-t diagram: Red shock (rising edge), green rarefaction fan (falling edge)")
print("  • Breakthrough: Sharp rise (shock), smooth decline (rarefaction)")
print("  • Demonstrates classic rarefaction wave behavior")

## Example 2: Multi-Step Input with Complex Wave Interactions

Multiple concentration increases ($0 \to 10 \to 15 \to 20$) demonstrate realistic wave interactions:
- Each concentration increase creates a shock
- Faster shocks (higher C) can catch slower shocks (lower C)
- Wave interactions create new waves
- Shows how complex inlet histories produce rich wave dynamics

This example illustrates the event-driven nature of front tracking and demonstrates that the solver handles multiple simultaneous waves correctly.

In [4]:
# Example 2 Setup
dates_ex2 = pd.date_range(start="2020-01-01", periods=100, freq="D")
tedges_ex2 = compute_time_edges(tedges=None, tstart=None, tend=dates_ex2, number_of_bins=len(dates_ex2))

# Multi-step increase: 0 → 10 → 15 → 20
cin_ex2 = np.zeros(len(dates_ex2))
cin_ex2[10:50] = 10.0  # First step: 0 → 10
cin_ex2[25:] = 15.0  # Second step: 10 → 15
cin_ex2[30:] = 20.0  # Third step: 15 → 20

# Aquifer properties
flow_ex2 = np.full(len(dates_ex2), 100.0)
aquifer_pore_volume_ex2 = 500.0  # m³

# Freundlich sorption (favorable)
freundlich_k_ex2 = 0.01
freundlich_n_ex2 = 2.0
bulk_density_ex2 = 1500.0
porosity_ex2 = 0.3

# Output grid
cout_dates_ex2 = pd.date_range(start=dates_ex2[0], periods=150, freq="D")
cout_tedges_ex2 = compute_time_edges(tedges=None, tstart=None, tend=cout_dates_ex2, number_of_bins=len(cout_dates_ex2))

print("Example 2: Multi-Step Concentration Increases")
print("  Step 1 (day 10): 0 → 10")
print("  Step 2 (day 25): 10 → 15")
print("  Step 3 (day 30): 15 → 20")
print(f"  Pore volume: {aquifer_pore_volume_ex2} m³, Flow: {flow_ex2[0]} m³/day")

Example 2: Multi-Step Concentration Increases
  Step 1 (day 10): 0 → 10
  Step 2 (day 25): 10 → 15
  Step 3 (day 30): 15 → 20
  Pore volume: 500.0 m³, Flow: 100.0 m³/day


In [ ]:
# Run simulation
cout_ex2, structure_ex2 = infiltration_to_extraction_front_tracking_detailed(
    cin=cin_ex2,
    flow=flow_ex2,
    tedges=tedges_ex2,
    cout_tedges=cout_tedges_ex2,
    aquifer_pore_volumes=aquifer_pore_volume_ex2,
    freundlich_k=freundlich_k_ex2,
    freundlich_n=freundlich_n_ex2,
    bulk_density=bulk_density_ex2,
    porosity=porosity_ex2,
)

# Print summary
print(
    f"\nResults: {structure_ex2[0]['n_events']} events | "
    f"{structure_ex2[0]['n_shocks']} shocks | "
    f"{structure_ex2[0]['n_rarefactions']} rarefactions | "
    f"First arrival: {structure_ex2[0]['t_first_arrival']:.1f} days"
)

# Verify physics
results_ex2 = verify_physics(structure_ex2, cout_ex2, cout_tedges_ex2, cin_ex2, verbose=False)
print(f"\n{results_ex2['summary']}")

# Create comprehensive visualization
axes_ex2 = plot_front_tracking_summary(
    structure_ex2,
    tedges_ex2,
    cin_ex2,
    cout_tedges_ex2,
    cout_ex2,
    t_max=100,
    show_events=True,
    show_inactive=True,
    title="Example 2: Multi-Step Input with Wave Interactions (n=2.0)",
)
plt.show()

print("\nKey Observations:")
print(f"  • {structure_ex2[0]['n_events']} wave interaction events in the domain")
print(f"  • {structure_ex2[0]['n_shocks']} shocks created from concentration increases")
print("  • V-t diagram shows event markers (x) where waves interact")
print("  • Final plateau at C=20 matches inlet maximum")

## Example 3: Unfavorable Sorption and Mirror-Image Behavior

With **unfavorable sorption** ($n < 1$), the physics reverses compared to favorable sorption:
- **Higher concentrations travel SLOWER** (higher retardation)
- **Lower concentrations travel FASTER** (lower retardation)

To demonstrate rarefactions with unfavorable sorption, we use a **concentration dip** (negative pulse):
- **Baseline**: $C = 10$ mg/L
- **Dip to**: $C = 2$ mg/L (days 40-100)
- **Return to baseline**: $C = 10$ mg/L

### Wave Formation
- **Leading edge** ($10 \to 2$): Concentration decreases → Slow (high C) behind fast (low C) → **RAREFACTION**
- **Trailing edge** ($2 \to 10$): Concentration increases → Fast (low C) catches slow (high C) → **SHOCK**

This is the **exact opposite** of favorable sorption (Example 1)!

In [6]:
# Example 3 Setup
dates_ex3 = pd.date_range(start="2020-01-01", periods=200, freq="D")
tedges_ex3 = compute_time_edges(tedges=None, tstart=None, tend=dates_ex3, number_of_bins=len(dates_ex3))

# Concentration dip: 10 → 2 → 10
cin_ex3 = np.full(len(dates_ex3), 10.0)  # Baseline: 10 mg/L
cin_ex3[:2] = 0.0  # Initial condition check
cin_ex3[40:100] = 2.0  # Dip to 2 mg/L

# Aquifer properties
flow_ex3 = np.full(len(dates_ex3), 100.0)
aquifer_pore_volume_ex3 = 200.0

# Freundlich sorption (UNFAVORABLE: n < 1)
freundlich_k_ex3 = 0.001  # Increased for visible nonlinearity
freundlich_n_ex3 = 0.7  # n < 1: Higher C is SLOWER
bulk_density_ex3 = 1500.0
porosity_ex3 = 0.3

# Output grid
cout_dates_ex3 = pd.date_range(start=dates_ex3[0], periods=250, freq="D")
cout_tedges_ex3 = compute_time_edges(tedges=None, tstart=None, tend=cout_dates_ex3, number_of_bins=len(cout_dates_ex3))

print("Example 3: Concentration Dip (Unfavorable Sorption)")
print("  Inlet: 10 → 2 (day 40) → 10 (day 100)")
print(f"  Freundlich: n={freundlich_n_ex3} (UNFAVORABLE), k={freundlich_k_ex3}")
print("  Physics: High C = High retardation (SLOW)")
print("  Expected: Rarefaction on leading edge (10→2), shock on trailing edge (2→10)")

Example 3: Concentration Dip (Unfavorable Sorption)
  Inlet: 10 → 2 (day 40) → 10 (day 100)
  Freundlich: n=0.7 (UNFAVORABLE), k=0.001
  Physics: High C = High retardation (SLOW)
  Expected: Rarefaction on leading edge (10→2), shock on trailing edge (2→10)


In [ ]:
# Run simulation
cout_ex3, structure_ex3 = infiltration_to_extraction_front_tracking_detailed(
    cin=cin_ex3,
    flow=flow_ex3,
    tedges=tedges_ex3,
    cout_tedges=cout_tedges_ex3,
    aquifer_pore_volumes=aquifer_pore_volume_ex3,
    freundlich_k=freundlich_k_ex3,
    freundlich_n=freundlich_n_ex3,
    bulk_density=bulk_density_ex3,
    porosity=porosity_ex3,
)

# Print summary
print(
    f"\nResults: {structure_ex3[0]['n_events']} events | "
    f"{structure_ex3[0]['n_shocks']} shocks | "
    f"{structure_ex3[0]['n_rarefactions']} rarefactions | "
    f"First arrival: {structure_ex3[0]['t_first_arrival']:.1f} days"
)

# Verify physics
results_ex3 = verify_physics(structure_ex3, cout_ex3, cout_tedges_ex3, cin_ex3, verbose=True)
print(f"\n{results_ex3['summary']}")

# Create comprehensive visualization
axes_ex3 = plot_front_tracking_summary(
    structure_ex3,
    tedges_ex3,
    cin_ex3,
    cout_tedges_ex3,
    cout_ex3,
    t_max=250,
    title="Example 3: Dip with Unfavorable Sorption (n=0.7) - Rarefaction on Leading Edge",
)
plt.show()

print("\nKey Features:")
print("  • V-t diagram: GREEN rarefaction fan on leading edge (10→2)")
print("  • Breakthrough: SMOOTH gradual decline (rarefaction), then SHARP rise (shock)")
print("  • This is the MIRROR IMAGE of favorable sorption (Example 1)")

### Comparison: How Sorption Type Affects Each Inlet

The comparison below shows how **the same inlet timeseries** produces **different outlet concentrations** with favorable vs unfavorable sorption. We run each inlet (pulse and dip) with both sorption types to demonstrate the mirror-image behavior:

**The Comparison Will Show (2×3 grid):**
- **Row 1**: Pulse inlet (0→10→0) → outlets with favorable and unfavorable sorption
- **Row 2**: Dip inlet (10→2→10) → outlets with favorable and unfavorable sorption

This demonstrates that wave structure depends critically on the sorption isotherm, not just the inlet pattern!

In [ ]:
# Run cross-comparisons to complete the analysis
# We already have: pulse+favorable (ex1), dip+unfavorable (ex3)
# We need: pulse+unfavorable, dip+favorable

print("Running pulse inlet with unfavorable sorption (n=0.7)...")
_, structure_pulse_unfav = infiltration_to_extraction_front_tracking_detailed(
    cin=cin_ex1,
    flow=flow_ex1,
    tedges=tedges_ex1,
    cout_tedges=cout_tedges_ex1,
    aquifer_pore_volumes=aquifer_pore_volume_ex1,
    freundlich_k=freundlich_k_ex3,
    freundlich_n=freundlich_n_ex3,
    bulk_density=bulk_density_ex1,
    porosity=porosity_ex1,
)

print("Running dip inlet with favorable sorption (n=2.0)...")
_, structure_dip_fav = infiltration_to_extraction_front_tracking_detailed(
    cin=cin_ex3,
    flow=flow_ex3,
    tedges=tedges_ex3,
    cout_tedges=cout_tedges_ex3,
    aquifer_pore_volumes=aquifer_pore_volume_ex3,
    freundlich_k=freundlich_k_ex1,
    freundlich_n=freundlich_n_ex1,
    bulk_density=bulk_density_ex3,
    porosity=porosity_ex3,
)

# Create comprehensive comparison plot
fig_comp, axes_comp = plot_sorption_comparison(
    pulse_favorable_structure=structure_ex1,
    pulse_unfavorable_structure=structure_pulse_unfav,
    pulse_tedges=tedges_ex1,
    pulse_cin=cin_ex1,
    dip_favorable_structure=structure_dip_fav,
    dip_unfavorable_structure=structure_ex3,
    dip_tedges=tedges_ex3,
    dip_cin=cin_ex3,
    t_max_pulse=150,
    t_max_dip=250,
)
plt.show()

print("\n" + "=" * 80)
print("KEY INSIGHT: Same inlet → Opposite wave structures with different sorption!")
print("=" * 80)
print("Pulse (0→10→0):  Favorable = Shock↑ Rarefaction↓  |  Unfavorable = Rarefaction↑ Shock↓")
print("Dip (10→2→10):   Favorable = Rarefaction↓ Shock↑  |  Unfavorable = Shock↓ Rarefaction↑")
print("=" * 80)

## Summary: Rarefaction Waves in Front Tracking

### When Rarefactions Form

**Favorable Sorption ($n > 1$):**
- High C travels FAST (low retardation)
- Concentration **DECREASES** (nonzero to nonzero) → Rarefaction (expansion)
- Concentration **INCREASES** → Shock (compression)
- Example: Pulse trailing edge, step down

**Unfavorable Sorption ($n < 1$):**
- High C travels SLOW (high retardation)
- Concentration **INCREASES** (nonzero to nonzero) → Rarefaction (expansion)
- Concentration **DECREASES** → Shock (compression)
- Example: Dip leading edge

### Critical Implementation Details

1. **C=0 Transitions**: Transitions to/from C=0 create characteristics (not rarefactions/shocks) because C=0 represents the unknown initial condition
2. **V-t Diagrams**: Rarefactions appear as green fans expanding from origin points
3. **Breakthrough Curves**: Rarefactions create SMOOTH transitions (not sharp jumps)
4. **Exact Analytical**: All rarefaction concentrations computed using closed-form self-similar solutions: $R(C) = \frac{Q(t-t_0)}{V-V_0}$
5. **No Numerical Artifacts**: Zero numerical dispersion, no tolerance-based approximations
8. Exact mass balance satisfied (using analytical spatial integration)

### Key Observations from Examples

- **Example 1**: Classic rarefaction demonstration with favorable sorption
- **Example 2**: Complex wave interactions with multiple concentration steps
- **Example 3**: Unfavorable sorption shows mirror-image behavior
- **Comparison**: Directly illustrates how sorption type reverses wave structure

### Physics Verification

All examples passed comprehensive physics checks:
1. Shock entropy conditions satisfied
2. No negative concentrations
3. Output ≤ input maximum
4. Finite first arrival times
5. No NaN values after spin-up
6. Events chronologically ordered
7. Rarefaction wave ordering correct
8. Exact mass balance satisfied (using analytical spatial integration)

The front-tracking method provides exact analytical solutions with machine precision (rtol ~ 1e-14), making it ideal for validating numerical transport solvers.